In [12]:
# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True) # 요금 누락값 0원으로 채우기
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# LABEL ENCODING
def format_features(df):
    from sklearn.preprocessing import LabelEncoder
    df['Cabin'] = df['Cabin'].str[:1] # 승선실번호 앞 글자만 따옴
    features = ['Cabin', 'Sex', 'Embarked'] # 컬럼 이름을 리스트목록으로 저장
    for feature in features:
        le = LabelEncoder() # LabelEncoder 객체 생성
        # le = le.fit(df[feature]) # 들어오는 데이터프레임의 목록을 받아 학습
        # df[feature] = le.transform(df[feature]) # 숫자로 변환
        df[feature] = le.fit_transform(df[feature]) # 위 두 줄을 함축
        print(le.classes_) # 인코딩 속성값
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features():
    import pandas as pd
    df = pd.read_csv('train.csv')
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [13]:
titanic_df = transform_features()

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
['female' 'male']
['C' 'N' 'Q' 'S']


In [14]:
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0
2,1,3,0,26.0,0,0,7.9250,7,3
3,1,1,0,35.0,1,0,53.1000,2,3
4,0,3,1,35.0,0,0,8.0500,7,3


In [23]:
# from sklearn.preprocessing import LabelEncoder

In [ ]:
# encoder = LabelEncoder()

In [28]:
# encoder.inverse_transform(titanic_df['Sex'])

NameError: name 'inverse_transform' is not defined

In [29]:
from sklearn.model_selection import train_test_split

## 데이터 분리

In [32]:
X_df = titanic_df.drop(columns='Survived') # 입력
y_df = titanic_df['Survived'] # 정답

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_df,
                                                    y_df,
                                                    test_size=0.2,
                                                    random_state=11)

In [35]:
# 라이브러리 임포트
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [36]:
# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 모델 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')
# solver : 로지스틱 회귀의 최적화 알고리즘을 liblinear로 설정
# 일반적으로 작은 데이터 셋에서의 이진 분류는 liblinear가 성능이 약간 더 좋은 경향이 있다.

In [38]:
# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print(f'DecisionTreeClassifier 정확도: {accuracy_score(y_test, dt_pred)}')

DecisionTreeClassifier 정확도: 0.7877094972067039


In [39]:
# RandomForestClassifier 학습/예측/평가 : DecisionTreeClassifier 100개 모여서 학습함
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print(f'RandomForestClassifier 정확도: {accuracy_score(y_test, rf_pred)}')

RandomForestClassifier 정확도: 0.8547486033519553


In [41]:
# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print(f'LogisticRegression 정확도: {accuracy_score(y_test, lr_pred):.4f}')

LogisticRegression 정확도: 0.8659


## 교차검증

In [50]:
# 폴드 셋가 5개인 KFold 객체 생성,
# 폴드 수만큼 예측결과 저장을 위한 리스트 객체 생성
def exec_kfold(clf, folds=5): # 지정매개변수 folds=5
    from sklearn.model_selection import KFold
    import numpy as np
    kfold = KFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행
    for iter_count, (train_index, test_index) in enumerate(kfold.split(X_df)):
        # 데이터, 인덱스값이 같이 필요할 때 enumerate
        # X_titanic_df 데이터에서 교차 검증별 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X_df.values[train_index], X_df.values[test_index]
        y_train, y_test = y_df.values[train_index], y_df.values[test_index]
        # 컬럼과 인덱스를 제외한 값만 뽑는다.
        # Classifier 학습/예측/정확도 계산
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy) # 예측결과를 scores 리스트에 저장
        print(f'교차 검증 {iter_count+1} 정확도: {accuracy:.4f}')
        
    # 5개 폴드에서의 평균 정확도 계산
    print(f'평균 정확도: {np.mean(scores):.4}')

In [57]:
exec_kfold(dt_clf) # DecisionTreeClassifier 정확도: 0.7877094972067039

교차 검증 1 정확도: 0.7542
교차 검증 2 정확도: 0.7809
교차 검증 3 정확도: 0.7865
교차 검증 4 정확도: 0.7697
교차 검증 5 정확도: 0.8202
평균 정확도: 0.7823


In [58]:
exec_kfold(rf_clf) # RandomForestClassifier 정확도: 0.8547486033519553

교차 검증 1 정확도: 0.7933
교차 검증 2 정확도: 0.8090
교차 검증 3 정확도: 0.8371
교차 검증 4 정확도: 0.7753
교차 검증 5 정확도: 0.8596
평균 정확도: 0.8148


In [59]:
exec_kfold(lr_clf) # LogisticRegression 정확도: 0.8659

교차 검증 1 정확도: 0.7933
교차 검증 2 정확도: 0.7921
교차 검증 3 정확도: 0.7753
교차 검증 4 정확도: 0.7472
교차 검증 5 정확도: 0.8427
평균 정확도: 0.7901


In [ ]:
# 교차검증을 했을 때 실제 적용했을 때와 비슷한 결과를 얻을 수 있다.

## cross_val_score 교차검증 API
StratifiedKFold를 이용해 폴드 세트를 분할한다.

In [62]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [61]:
cross_val_score(dt_clf, X_df, y_df, cv=5)

array([0.74301676, 0.7752809 , 0.79213483, 0.78651685, 0.84269663])

In [64]:
# DecisionTreeClassifier
score = cross_val_score(dt_clf, X_df, y_df, cv=5)
print(f'개별정확도: {score}, 평균 정확도: {np.mean(score)}')
# 교차 검증 1 정확도: 0.7542
# 교차 검증 2 정확도: 0.7809
# 교차 검증 3 정확도: 0.7865
# 교차 검증 4 정확도: 0.7697
# 교차 검증 5 정확도: 0.8202
# 평균 정확도: 0.7823

개별정확도: [0.74301676 0.7752809  0.79213483 0.78651685 0.84269663], 평균 정확도: 0.7879291946519366


In [65]:
# RandomForestClassifier
score = cross_val_score(rf_clf, X_df, y_df, cv=5)
print(f'개별정확도: {score}, 평균 정확도: {np.mean(score)}')
# 교차 검증 1 정확도: 0.7933
# 교차 검증 2 정확도: 0.8090
# 교차 검증 3 정확도: 0.8371
# 교차 검증 4 정확도: 0.7753
# 교차 검증 5 정확도: 0.8596
# 평균 정확도: 0.8148

개별정확도: [0.79329609 0.79775281 0.84831461 0.76404494 0.86516854], 평균 정확도: 0.8137153976523758


In [66]:
# LogisticRegression
score = cross_val_score(lr_clf, X_df, y_df, cv=5)
print(f'개별정확도: {score}, 평균 정확도: {np.mean(score)}')
# 교차 검증 1 정확도: 0.7933
# 교차 검증 2 정확도: 0.7921
# 교차 검증 3 정확도: 0.7753
# 교차 검증 4 정확도: 0.7472
# 교차 검증 5 정확도: 0.8427
# 평균 정확도: 0.7901

개별정확도: [0.7877095  0.79213483 0.7752809  0.76404494 0.82022472], 평균 정확도: 0.7878789780930262


# GridSearchCV
파라미터 튜닝까지 한 번에.

### DecisionTreeClassifier GridSearchCV

In [67]:
from sklearn.model_selection import GridSearchCV
# 파라미터 값이 달라서 로지스틱회귀는 할 수 없다.

In [68]:
parameters = {'max_depth':[2, 3, 5, 10],
              'min_samples_split':[2, 3, 5],
              'min_samples_leaf':[1, 5, 8]}

In [71]:
grid_clf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5) # 객체 생성
grid_clf.fit(X_train, y_train) # 학습

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=11),
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 8],
                         'min_samples_split': [2, 3, 5]},
             scoring='accuracy')

In [73]:
print('GridSearchCV 최적 하이퍼 파라미터 :', grid_clf.best_params_)
print(f'GridSearchCV 최고 정확도 : {grid_clf.best_score_:.4f}')

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.7992


In [75]:
pred = grid_clf.best_estimator_.predict(X_test) # 하이퍼 파라미터로 학습된 Estimator로 예측

In [76]:
accuracy_score(y_test, pred) # 검증

0.8715083798882681

### RandomForestClassifier GridSearchCV

In [79]:
grid_clf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5) # 객체 생성
grid_clf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :', grid_clf.best_params_)
print(f'GridSearchCV 최고 정확도 : {grid_clf.best_score_:.4f}')

pred = grid_clf.best_estimator_.predict(X_test)

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.8146


In [80]:
accuracy_score(y_test, pred)

0.88268156424581

In [ ]:
# 단순 정확도로만 판단하기에는 상황에 따라 신빙성이 없을 수 있다.